### **All Slack Starting Method**

In [22]:
import copy

import numpy as np
from tabulate import tabulate

In [25]:
def visualize_tabulation(tabulation, all_vars, basic_vars):

    header = np.concatenate(([], all_vars, ["sol"]))
    print(tabulate(np.concatenate((np.array(basic_vars).reshape(-1, 1), tabulation), axis=1), headers=header, stralign='center', tablefmt='simple_grid'))

In [56]:
# question 4
# no. of decision variables (N)
NO_OF_DECISION_VARS = 3

# no. of conditions 
NO_OF_CONDITIONS = 3

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [2.35, 3, 2.85]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MAX" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, <solution>]
conditions = [
    [ 10, 9, 14, 4_000],
    [  5, 6, 12, 3_000],
    [  0, 1,  0,   250],
]

### Creating the Variables

In [57]:
var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S"

# ========================= ALL THE VARIABLES =========================
var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

for i in range(NO_OF_CONDITIONS):
    var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("variables:", var_symbol_arr)

# ========================== BASIC VARIABLES ==========================
basic_var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_CONDITIONS):
    basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

print("basic variables:", basic_var_symbol_arr)

variables: ['P', 'X1', 'X2', 'X3', 'S1', 'S2', 'S3']
basic variables: ['P', 'S1', 'S2', 'S3']


### Creating the Initial Tabulation

In [58]:
# verify the elements
OBJ_FUNC = np.array(OBJ_FUNC)
if len(OBJ_FUNC) != NO_OF_DECISION_VARS:
    raise Exception(f"the no. of coefficients in the objective function doesn't match tne number of decision variables...")

conditions = np.array(conditions)
if len(conditions) != NO_OF_CONDITIONS:
    raise Exception(f"the no. of specified conditions doesn't match 'NO_OF_CONDITIONS'...")
if len(conditions[0]) != NO_OF_DECISION_VARS + 1:
    raise Exception(f"the no. of coefficients in the conditions doesn't match tne number of decision variables...")

In [59]:
# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + NO_OF_CONDITIONS + 1

# find the column size
col_size = 1 + NO_OF_CONDITIONS # no. of basic vars = 1 (for objective function) + no. of conditions

# construct the initial tabulation
obj_func_row = np.concatenate(([1], -OBJ_FUNC, np.zeros([NO_OF_CONDITIONS]), [0]))
initial_tabulation = obj_func_row.reshape((1, -1))

for i in range(1, col_size):

    # condition rows
    temp = np.zeros([NO_OF_CONDITIONS])
    temp[i-1] = 1
    row = np.concatenate(([0], conditions[i-1][ :-1], temp, conditions[i-1][-1: ])).reshape((1, -1))

    initial_tabulation = np.concatenate((initial_tabulation, row), axis=0)

visualize_tabulation(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

┌────┬─────┬───────┬──────┬───────┬──────┬──────┬──────┬───────┐
│    │   P │    X1 │   X2 │    X3 │   S1 │   S2 │   S3 │   sol │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ P  │   1 │ -2.35 │   -3 │ -2.85 │    0 │    0 │    0 │     0 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │ 10    │    9 │ 14    │    1 │    0 │    0 │  4000 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │  5    │    6 │ 12    │    0 │    1 │    0 │  3000 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S3 │   0 │  0    │    1 │  0    │    0 │    0 │    1 │   250 │
└────┴─────┴───────┴──────┴───────┴──────┴──────┴──────┴───────┘


### Performing One Iteration of Optimization

In [16]:
# ====================================== ENTERING VARIABLE ======================================
# find the highest negative value in the objective function row -> pivot column
tabulation = copy.deepcopy(initial_tabulation)

if OPT_TYPE == "MAX":
    if not (tabulation[0][1:-1] < 0).any():
        raise Exception(f"there are no negative values in the objective row...")
    pivot_col_value = np.min(tabulation[0][1:-1]) # highest negative value
else: # OPT_TYPE == "MIN"
    if not (tabulation[0][1:-1] > 0).any():
        raise Exception(f"there are no positive values in the objective row...")
    pivot_col_value = np.max(tabulation[0][1:-1]) # highest positive value
pivot_col_idx = list(tabulation[0][1:-1]).index(pivot_col_value) + 1

entering_var = var_symbol_arr[pivot_col_idx]
print(f"entering var: {entering_var}")
pivot_col = tabulation[:, pivot_col_idx]
print(f"pivot col: {pivot_col}")

# find the ratio column
solution  = tabulation[:, -1]
print(f"solution : {solution}")
ratio_col = solution / pivot_col
print(f"ratio col: {ratio_col}")

# ====================================== LEAVING VARIABLE =======================================
# find the lowest positive value in the ratio column
pivot_row_value = np.min(ratio_col[1: ][ratio_col[1: ] > 0])
pivot_row_idx = list(ratio_col[1: ]).index(pivot_row_value) + 1

leaving_var = basic_var_symbol_arr[pivot_row_idx]
print(f"leaving var : {leaving_var}")
pivot_row = tabulation[pivot_row_idx]
print(f"pivot row: {pivot_row}")

entering var: X1
pivot col: [-5.  6.  1. -1.  0.]
solution : [ 0. 24.  6.  1.  2.]
ratio col: [-0.  4.  6. -1. inf]
leaving var : S1
pivot row: [ 0.  6.  4.  1.  0.  0.  0. 24.]


/tmp/ipykernel_400753/1569952453.py:23: RuntimeWarning: divide by zero encountered in divide
  ratio_col = solution / pivot_col


In [17]:
# replace the leaving variable with the entering variable
basic_var_symbol_arr[pivot_row_idx] = entering_var
print(f"new basic variables: {basic_var_symbol_arr}")

new basic variables: ['P', 'X1', 'S2', 'S3', 'S4']


In [18]:
# convert the pivot column to a pivot vector

# verify that the pivot column is convertible to a pivot vector
if pivot_row[pivot_col_idx] == 0:
    raise Exception(f"the pivot column (var '{entering_var}') at index {pivot_col_idx} is not convertible to a pivot vector...")

# transform the pivot row
pivot_row_ = pivot_row / pivot_row[pivot_col_idx]
tabulation[pivot_row_idx] = pivot_row_

# transform all the other rows
for i in range(col_size):

    if i == pivot_row_idx:
        continue

    row = tabulation[i]
    tabulation[i] = row - row[pivot_col_idx] * pivot_row_

visualize_tabulation(tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

┌────┬─────┬──────┬───────────┬───────────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │        X2 │        S1 │   S2 │   S3 │   S4 │   sol │
├────┼─────┼──────┼───────────┼───────────┼──────┼──────┼──────┼───────┤
│ P  │   1 │    0 │ -0.666667 │  0.833333 │    0 │    0 │    0 │    20 │
├────┼─────┼──────┼───────────┼───────────┼──────┼──────┼──────┼───────┤
│ X1 │   0 │    1 │  0.666667 │  0.166667 │    0 │    0 │    0 │     4 │
├────┼─────┼──────┼───────────┼───────────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │    0 │  1.33333  │ -0.166667 │    1 │    0 │    0 │     2 │
├────┼─────┼──────┼───────────┼───────────┼──────┼──────┼──────┼───────┤
│ S3 │   0 │    0 │  1.66667  │  0.166667 │    0 │    1 │    0 │     5 │
├────┼─────┼──────┼───────────┼───────────┼──────┼──────┼──────┼───────┤
│ S4 │   0 │    0 │  1        │  0        │    0 │    0 │    1 │     2 │
└────┴─────┴──────┴───────────┴───────────┴──────┴──────┴──────┴───────┘


### Checking the Feasibility Condition

In [19]:
# the values in the solution column except the one in the objective function row must be non-negative
is_feasible = (tabulation[:, -1][1: ] >= 0).all()

if not is_feasible:
    raise Exception(f"the tabulation is not feasible...")
print(f"the tabulation is feasible.")

the tabulation is feasible.


### Checking the Optimality Condition

- The optimality condition is embedded in creating the initial tabulation. 

### Constructing the Code into a Loop

In [60]:
iteration_no = 0
tabulation = copy.deepcopy(initial_tabulation)

for _ in range(5): # *****

    # ====================================== ENTERING VARIABLE ======================================
    if OPT_TYPE == "MAX":
        if not (tabulation[0][1:-1] < 0).any():
            # raise Exception(f"there are no negative values in the objective row...")
            print(f"*****there are no more negative values in the objective row; tabulation is optimal...*****\n")
            break
        pivot_col_value = np.min(tabulation[0][1:-1]) # highest negative value
    else: # OPT_TYPE == "MIN"
        if not (tabulation[0][1:-1] > 0).any():
            # raise Exception(f"there are no positive values in the objective row...")
            print(f"*****there are no more positive values in the objective row; tabulation is optimal...*****\n")
            break
        pivot_col_value = np.max(tabulation[0][1:-1]) # highest positive value

    iteration_no += 1
    print("="*20 + f" iteration no.: {iteration_no} " + "="*20)

    pivot_col_idx = list(tabulation[0][1:-1]).index(pivot_col_value) + 1

    entering_var = var_symbol_arr[pivot_col_idx]
    print(f"entering var: {entering_var}")
    pivot_col = tabulation[:, pivot_col_idx]
    print(f"pivot col: {pivot_col}")

    # find the ratio column
    solution  = tabulation[:, -1]
    print(f"solution : {solution}")
    ratio_col = solution / pivot_col
    print(f"ratio col: {ratio_col}")

    # ====================================== LEAVING VARIABLE =======================================
    # find the lowest positive value in the ratio column
    if not (ratio_col[1: ] > 0).any():
        raise Exception(f"there is no positive value in the ratio column; therefore, NO FEASIBLE SOLUTION EXISTS...")

    pivot_row_value = np.min(ratio_col[1: ][ratio_col[1: ] > 0])
    pivot_row_idx = list(ratio_col[1: ]).index(pivot_row_value) + 1

    leaving_var = basic_var_symbol_arr[pivot_row_idx]
    print(f"leaving var : {leaving_var}")
    pivot_row = tabulation[pivot_row_idx]
    print(f"pivot row: {pivot_row}")

    # replace the leaving variable with the entering variable
    basic_var_symbol_arr[pivot_row_idx] = entering_var
    print(f"new basic variables: {basic_var_symbol_arr}")

    # convert the pivot column to a pivot vector

    # verify that the pivot column is convertible to a pivot vector
    if pivot_row[pivot_col_idx] == 0:
        raise Exception(f"the pivot column (var '{entering_var}') at index {pivot_col_idx} is not convertible to a pivot vector...")

    # transform the pivot row
    pivot_row_ = pivot_row / pivot_row[pivot_col_idx]
    tabulation[pivot_row_idx] = pivot_row_

    # transform all the other rows
    for i in range(col_size):

        if i == pivot_row_idx:
            continue

        row = tabulation[i]
        tabulation[i] = row - row[pivot_col_idx] * pivot_row_

    visualize_tabulation(tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

    # the values in the solution column except the one in the objective function row must be non-negative
    is_feasible = (tabulation[:, -1][1: ] >= 0).all()

    if not is_feasible:
        raise Exception(f"the tabulation is not feasible...")
    print(f"the tabulation is feasible.")

    print('\n')

# check for many solutions
non_basic_vars = list(set(var_symbol_arr) - set(basic_var_symbol_arr))
non_basic_vars.sort()
for non_basic_var in non_basic_vars:
    # find the objective row value for each non-basic variable
    # print(non_basic_var, tabulation[0][var_symbol_arr.index(non_basic_var)])
    if tabulation[0][var_symbol_arr.index(non_basic_var)] == 0:
        raise Exception(f"non-basic variable '{non_basic_var}' has a zero value in the objective row; therefore, MANY SOLUTIONS EXISTS...")

==================== iteration no.: 1 ====================
entering var: X2
pivot col: [-3.  9.  6.  1.]
solution : [   0. 4000. 3000.  250.]
ratio col: [ -0.         444.44444444 500.         250.        ]
leaving var : S3
pivot row: [  0.   0.   1.   0.   0.   0.   1. 250.]
new basic variables: ['P', 'S1', 'S2', 'X2']
┌────┬─────┬───────┬──────┬───────┬──────┬──────┬──────┬───────┐
│    │   P │    X1 │   X2 │    X3 │   S1 │   S2 │   S3 │   sol │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ P  │   1 │ -2.35 │    0 │ -2.85 │    0 │    0 │    3 │   750 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │ 10    │    0 │ 14    │    1 │    0 │   -9 │  1750 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │  5    │    0 │ 12    │    0 │    1 │   -6 │  1500 │
├────┼─────┼───────┼──────┼───────┼──────┼──────┼──────┼───────┤
│ X2 │   0 │  0    │    1 │  0    │    0 │    0 │    1 │   250 │
└────┴─────┴───────┴──────┴─

/tmp/ipykernel_400753/1391603999.py:33: RuntimeWarning: divide by zero encountered in divide
  ratio_col = solution / pivot_col
